In [1]:
import pandas as pd
import numpy as np
import math
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.utils.validation import _check_large_sparse
from sklearn.metrics import pairwise_distances_argmin_min, jaccard_score, f1_score
from sklearn.model_selection import cross_val_score, KFold
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier
from sklearn.preprocessing import MinMaxScaler
from sklearn.multioutput import ClassifierChain
from sklearn import svm
from sklearn.metrics import multilabel_confusion_matrix, ConfusionMatrixDisplay

In [2]:
def boolean_df(item_lists, unique_items):
# Create empty dict
    bool_dict = {}
    
    # Loop through all the tags
    for i, item in enumerate(unique_items):
        
        # Apply boolean mask
        bool_dict[item] = item_lists.apply(lambda x: 1 if item in x else 0)
            
    # Return the results as a dataframe
    return pd.DataFrame(bool_dict)

def to_1D(series):
    return pd.Series([x for _list in series for x in _list])

Edit the cell below to load another dataset or combine them together.

In [47]:
# Reading NA data
X_df = pd.read_csv('../new_data/Persian/persian_dataset.csv', index_col=None)
Y_df = pd.read_csv('../new_data/Persian/p_labels_cleaned.csv', usecols=['filename', 'emotions'], index_col='filename')
# Reading Persian data
# X_df_p = pd.read_csv('../new_data/Persian/persian_dataset.csv', index_col=None)
# Y_df_p = pd.read_csv('../new_data/Persian/p_labels_cleaned.csv', usecols=['filename', 'emotions'], index_col='filename')

# concatenate them
# Y_df = pd.concat([Y_df_na, Y_df_p], sort=False)
# X_df = pd.concat([X_df_na, X_df_p], ignore_index=True, sort=False)


In [48]:
# Remove videos that we deleted from labels.
X_df = X_df[X_df['filename'].isin(Y_df.index)]
# Reset index to prevent future errors in using iloc
X_df.reset_index(inplace=True, drop=True)
# Change emotion column to list type
Y_df["emotions"] = Y_df["emotions"].apply(eval)

label_cols = to_1D(Y_df["emotions"]).unique() 
labels_expanded = boolean_df(Y_df['emotions'], label_cols )


In [49]:
X_df['none']  = np.NaN
X_df['furious']  = np.NaN
X_df['anger']  = np.NaN
X_df['annoyed']  = np.NaN
X_df['contempt']  = np.NaN
X_df['hatred']  = np.NaN


In [ ]:
X_df.head()

In [50]:
for index, row in X_df.iterrows():
    # print(index, row)
    filename = X_df.iloc[index]['filename']
    # print(labels_expanded.loc[filename]['none':'hatred'].to_list())
    X_df.at[index,'none'] = labels_expanded.at[filename,'none']
    X_df.at[index,'furious'] = labels_expanded.at[filename,'furious']
    X_df.at[index,'anger'] = labels_expanded.at[filename,'anger']
    X_df.at[index,'annoyed'] = labels_expanded.at[filename,'annoyed']
    X_df.at[index,'contempt'] = labels_expanded.at[filename,'contempt']
    X_df.at[index,'hatred'] = labels_expanded.at[filename,'hatred']

## Ablation Studies
Run the cell below to remove columns for ablation studies and then run the training cell.

In [62]:
label_cols = ['none', 'furious', 'anger', 'annoyed', 'contempt', 'hatred']
frames = X_df[X_df['none'] == 1].shape[0]
print("Ratio: ", frames/X_df.shape[0])
label_ratio = {'anger': 0.50, 'contempt':0.30, 'hatred': 0.30, 'annoyed': 0.30, 'furious': 0.35, 'none':0.30} # For NA

Ratio:  0.13832948167712117


In [ ]:
## Ablation cols
# ablation_cols = ['AU01_r','AU02_r','AU04_r','AU05_r','AU06_r','AU07_r','AU09_r', 'AU10_r','AU12_r','AU14_r','AU15_r','AU17_r','AU20_r','AU23_r','AU25_r','AU26_r','AU45_r']
ablation_cols = ['pose_Rx','pose_Ry','pose_Rz','gaze_angle_x','gaze_angle_y']

In [ ]:
X_df = X_df.drop(columns=ablation_cols)
X_df

### Min-Max Scaling
To limit the scale of features

In [15]:
cols_to_scale = list (
    set(X_df.columns.to_list()) - set(['frame', 'face_id', 'culture', 'filename', 'timestamp', 'confidence','success', 'none', 'furious', 'anger', 'annoyed', 'contempt', 'disgust', 'hatred'])
)
scaler = MinMaxScaler()
X_df[cols_to_scale] = scaler.fit_transform(X_df[cols_to_scale])

In [ ]:
X_df.tail()

## Splitting into train and test
Randomly select 25% of videos apart for testing.

In [69]:
videos = X_df['filename'].unique()
# Comment this line and uncomment next to test on Persian videos
# test_videos = ['na/vid_13.mp4', 'na/vid_92.mp4', 'na/vid_93.mp4', 'na/vid_6.mp4', 'na/vid_14.mp4', 'na/vid_10_1.mp4', 'na/vid_34.mp4', 'na/vid_86.mp4', 'na/vid_83.mp4', 'na/vid_90.mp4', 'na/vid_60.mp4', 'na/vid_24.mp4', 'na/vid_10_3.mp4', 'na/vid_10_2.mp4', 'na/vid_87.mp4', 'na/vid_32.mp4', 'na/vid_79.mp4', 'na/vid_68.mp4', 'na/vid_56.mp4', 'na/vid_36.mp4']
# test_videos = ['persian/vid_59.mp4', 'persian/vid_79.mp4', 'persian/vid_40.mp4', 'persian/vid_10.mp4', 'persian/vid_1.mp4', 'persian/vid_5.mp4', 'persian/vid_44.mp4', 'persian/vid_85.mp4', 'persian/vid_25.mp4', 'persian/vid_75.mp4', 'persian/vid_66.mp4', 'persian/vid_50.mp4', 'persian/vid_81.mp4', 'persian/vid_60.mp4', 'persian/vid_87.mp4', 'persian/vid_90.mp4', 'persian/vid_35.mp4', 'persian/vid_37.mp4', 'persian/vid_42.mp4', 'persian/vid_69.mp4', 'persian/vid_70.mp4', 'persian/vid_52.mp4', 'persian/vid_53.mp4', 'persian/vid_82.mp4']
test_videos = pd.Series(videos).sample(frac=0.25)
# print(test_videos.to_list())

In [70]:
metadata_cols = ['frame', 'face_id', 'culture', 'filename', 'timestamp']
label_cols = ['none', 'furious', 'anger', 'annoyed', 'contempt', 'hatred']

print(X_df.head())

train_videos = np.array(list(set(videos) - set(test_videos)))
test_df = X_df[X_df['filename'].isin(test_videos)]
metadata_test = test_df[metadata_cols]
metadata_test.reset_index(inplace=True)
y_test = test_df[label_cols].values
X_test = test_df.drop(columns = ['frame', 'face_id', 'culture', 'filename', 'timestamp', 'confidence','success'] + label_cols).values

            filename  culture  frame  face_id  timestamp  confidence  success  \
0  persian/vid_1.mp4  persian      1        0      0.000        0.98        1   
1  persian/vid_1.mp4  persian      2        0      0.033        0.98        1   
2  persian/vid_1.mp4  persian      3        0      0.067        0.98        1   
3  persian/vid_1.mp4  persian      4        0      0.100        0.98        1   
4  persian/vid_1.mp4  persian      5        0      0.133        0.98        1   

   AU01_r  AU02_r  AU04_r  ...  pose_Ry  pose_Rz  gaze_angle_x  gaze_angle_y  \
0    0.94    0.24    0.43  ...   -0.451   -0.106         0.408         0.466   
1    0.17    0.00    0.00  ...   -0.479   -0.113         0.478         0.490   
2    0.10    0.00    0.00  ...   -0.511   -0.112         0.494         0.471   
3    0.00    0.00    0.00  ...   -0.537   -0.112         0.540         0.466   
4    0.00    0.00    0.00  ...   -0.562   -0.115         0.581         0.467   

   none  furious  anger  annoyed

In [ ]:
y_test[800:805,:]


In [ ]:
metadata_test.iloc[800:805]

## Training with Cross-validation

### Classifier Chain
Choose one of the classifiers for CC: base_knn, base_xgb

In [74]:
from sklearn.model_selection import KFold
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn import metrics
from sklearn.multiclass import OneVsRestClassifier
from sklearn.multioutput import MultiOutputClassifier
from xgboost import XGBClassifier


kfold = KFold(5, True, 1)
frames_mean_hm_test = []
frames_mean_jac_test = []
videos_mean_jac_test = []
videos_mean_hm_test = []
# metadata_test.reset_index(inplace=True)

col_indices = {i:label for (i,label) in enumerate(label_cols)}
best_val = 0
best_model = None
splits = kfold.split(train_videos)
for (i, (train, test)) in enumerate(splits):
    # print(videos[train])
    # print(videos[test])
    print('%d-th split: train: %d, test: %d' % (i+1, len(train_videos[train]), len(train_videos[test])))
    train_df = X_df[X_df['filename'].isin(train_videos[train])]
    print(train_videos[train])
    train_metadata = train_df[metadata_cols]
    print('Training+validation data size: ', train_df.shape[0])
    y_train = train_df[label_cols].values
    X_train = train_df.drop(columns = ['frame', 'face_id', 'culture', 'filename', 'timestamp', 'confidence','success'] + label_cols).values
    valid_df = X_df[X_df['filename'].isin(train_videos[test])]
    y_valid = valid_df[label_cols].values
    X_valid = valid_df.drop(columns = ['frame', 'face_id', 'culture', 'filename', 'timestamp', 'confidence','success'] + label_cols).values
    # X_train, X_valid, y_train, y_valid = train_test_split(X, y)
    print('Training data size: ', X_train.shape[0])
    print('Validation data size: ', X_valid.shape[0])
    base_knn =  KNeighborsClassifier(n_neighbors=5,)
    base_xgb = XGBClassifier(objective="binary:logistic", learning_rate=0.2, eval_metric='logloss')
    base_rf = RandomForestClassifier()

    # ovr = MultiOutputClassifier(base_xgb)
    # ovr.fit(X_train, y_train)
    # valid_pred_ovr = ovr.predict(X_valid)
    # ovr_jaccard_score = jaccard_score(y_valid, valid_pred_ovr, average='samples')
    # ovr_ham_loss = metrics.hamming_loss(y_valid, valid_pred_ovr)
    # print("One-vs-Rest validation Jaccard score:" , ovr_jaccard_score)
    # print("One-vs-Rest validation Hamming loss: " , ovr_ham_loss)

   
    # Y_pred_ovr = ovr.predict(X_test)
    # a = jaccard_score(y_test, Y_pred_ovr, average='samples')
    # b = metrics.hamming_loss(y_test, Y_pred_ovr)
    # # print(Y_pred_ovr[800:805,:])
    # # print(y_test[800:805,:])

    # print("One-vs-Rest test Jaccard score: ", a)
    # print("One-vs-Rest test Hamming loss: " , b)
    
    
    chains = [ClassifierChain(base_knn, order='random', random_state=0) for i in range(7)]
    best_model_index = 0
    best_jac = 0            
    for j, model in enumerate(chains):
        model.fit(X_train, y_train)
        valid_pred = model.predict(X_valid)
        val_score =jaccard_score(y_valid, valid_pred, average='samples')
        if val_score > best_jac:
            best_model_index = j
            best_jac = val_score
            
        
    # predict on validation data
    valid_pred_chains = chains[best_model_index].predict(X_valid)
    chain_jaccard_scores = jaccard_score(y_valid, valid_pred_chains >= .5,
                                    average='samples')

    
    print("CC Validation Jaccard Score:\n ", chain_jaccard_scores)
    if (chain_jaccard_scores > best_val):
        best_val = chain_jaccard_scores
        best_model = chains[best_model_index]
    

# test on test data
Y_pred_chains = best_model.predict(X_test)
chain_jaccard_scores = jaccard_score(y_test, Y_pred_chains >= .5,
                                average='samples')
                
frames_mean_jac_test.append(np.mean(chain_jaccard_scores))
print("CC Test Jaccard Score: \n ", chain_jaccard_scores)

## voting predicted labels
# test_result_df = pd.DataFrame(columns=metadata_cols, data=metadata_test.values)
# test_result_df.update(metadata_test)
temp_df = pd.DataFrame(data=Y_pred_chains, columns=label_cols)
test_result_df = pd.concat([metadata_test, temp_df], axis=1)
video_groups = test_result_df.groupby('filename')[label_cols].sum()
ground_truth_video_labels = []
for v, row in video_groups.iterrows():
    # number of 1s in ground truth labels
    ground_truth_video_labels.append(test_df[test_df['filename'] == v].iloc[0][label_cols])
    
    # num_1s = test_df[test_df['filename'] == v].iloc[0][label_cols].sum()
    # num_1s = int(num_1s)
    # # ##### TODO: Bug is somewhere here:
    # a = np.argsort(video_groups.loc[v].values)
    # for i in range(len(a) - 1, len(a) - num_1s - 1, -1):
    #     video_groups.loc[v][a[i]] = 1
    # for i in range(0, len(a) - num_1s):
    #     video_groups.loc[v][a[i]] = 0

    # num_1s = test_df[test_df['filename'] == v].iloc[0][label_cols].sum()
    frames = test_df[test_df['filename'] == v].shape[0]
    # print(num_1s)
    for i in label_ratio.keys():
        prediction_ratio = row[i]/frames
        row[i] = 1 if prediction_ratio >= label_ratio[i] else 0
        # row[i] = prediction_ratio

    
# print(np.array(ground_truth_video_labels,  dtype=int).shape)
# print(video_groups.values.shape)

# j = metrics.jaccard_score(np.array(ground_truth_video_labels,  dtype=int), video_groups.values, average='samples')
# h = metrics.hamming_loss(np.array(ground_truth_video_labels,  dtype=int), video_groups.values)
# videos_mean_jac_test.append(j)
# videos_mean_hm_test.append(h)


1-th split: train: 54, test: 14
['persian/vid_78.mp4' 'persian/vid_15.mp4' 'persian/vid_64.mp4'
 'persian/vid_51.mp4' 'persian/vid_74.mp4' 'persian/vid_86.mp4'
 'persian/vid_29.mp4' 'persian/vid_2.mp4' 'persian/vid_96.mp4'
 'persian/vid_35.mp4' 'persian/vid_59.mp4' 'persian/vid_8.mp4'
 'persian/vid_89.mp4' 'persian/vid_37.mp4' 'persian/vid_52.mp4'
 'persian/vid_43.mp4' 'persian/vid_22.mp4' 'persian/vid_45.mp4'
 'persian/vid_6.mp4' 'persian/vid_79.mp4' 'persian/vid_82.mp4'
 'persian/vid_10.mp4' 'persian/vid_60.mp4' 'persian/vid_25.mp4'
 'persian/vid_16.mp4' 'persian/vid_50.mp4' 'persian/vid_13.mp4'
 'persian/vid_41.mp4' 'persian/vid_19.mp4' 'persian/vid_27.mp4'
 'persian/vid_85.mp4' 'persian/vid_23.mp4' 'persian/vid_56.mp4'
 'persian/vid_76.mp4' 'persian/vid_4.mp4' 'persian/vid_12.mp4'
 'persian/vid_88.mp4' 'persian/vid_94.mp4' 'persian/vid_70.mp4'
 'persian/vid_68.mp4' 'persian/vid_92.mp4' 'persian/vid_67.mp4'
 'persian/vid_66.mp4' 'persian/vid_91.mp4' 'persian/vid_48.mp4'
 'persian/vi

In [23]:
np.array(ground_truth_video_labels,  dtype=int)

array([], dtype=int64)

## Classification Report

In [75]:
print('------------------- Frame-level F1 -----------------')
print(metrics.classification_report(y_test, Y_pred_chains, target_names=label_cols))
print('------------------- Video-level F1 -----------------')
print(metrics.classification_report(np.array(ground_truth_video_labels,  dtype=int),  video_groups.values, target_names=label_cols))

------------------- Frame-level F1 -----------------
              precision    recall  f1-score   support

        none       0.37      0.13      0.19       721
     furious       0.60      0.37      0.46       825
       anger       0.47      0.46      0.46      1355
     annoyed       0.12      0.18      0.14       422
    contempt       0.33      0.28      0.30       630
      hatred       0.47      0.11      0.18      1005

   micro avg       0.40      0.28      0.33      4958
   macro avg       0.39      0.25      0.29      4958
weighted avg       0.43      0.28      0.32      4958
 samples avg       0.41      0.27      0.30      4958

------------------- Video-level F1 -----------------
              precision    recall  f1-score   support

        none       0.50      0.33      0.40         3
     furious       0.67      0.40      0.50         5
       anger       0.75      0.60      0.67        10
     annoyed       0.12      0.20      0.15         5
    contempt       0.29   

In [ ]:
print('------------------- Frame-level Jaccard -----------------')
print(metrics.jaccard_score(y_test, Y_pred_chains, average='samples'))
print('------------------- Video-level Jaccard -----------------')
print(metrics.jaccard_score(np.array(ground_truth_video_labels,  dtype=int),  video_groups.values, average='samples'))

Print average of metrics


In [ ]:
print("F-jac: ", np.mean(frames_mean_jac_test))
print("V-jac: ", np.mean(videos_mean_jac_test))
print("F-ham: ", np.mean(frames_mean_hm_test))
print("V-ham: ", np.mean(videos_mean_hm_test))

In [ ]:
videos_mean_jac_test

In [ ]:
print(test_videos.to_list())

## Confused frames
The cell below finds frames that are misclassified

In [ ]:

# Confusions
confused = 0
# for i,_ in enumerate(y_test):
for i in range(450, 550):
    if np.subtract(y_test[i], test_result_df.iloc[i][label_cols].to_numpy(dtype=int)).sum() != 0.0:
        confused += 1
        print(metadata_test.iloc[i], "------- Actual: ", y_test[i], " ----- pred: ", Y_pred_chains[i])
print("confused: ", confused)

In [ ]:
print("F-jac: ", np.mean(frames_mean_jac_test))
print("V-jac: ", np.mean(videos_mean_jac_test))
print("F-ham: ", np.mean(frames_mean_hm_test))
print("V-ham: ", np.mean(videos_mean_hm_test))

In [ ]:
fig, ax = plt.subplots(figsize=(4, 4))
ax.grid(True)
ax.set_title('CC Jaccard Score')
ax.set_xticks(range(0,7))
ax.set_xticklabels(range(0,7))
ax.set_ylabel('Jaccard Similarity Score')
ax.set_ylim([0.0, 1.1])

plt.plot(frames_mean_jac_test, label='Frames Jaccard Score', color='blue')
plt.plot(videos_mean_jac_test, label='Video Jaccard Score', color='green')
plt.legend(loc='best')
plt.show()

In [ ]:
fig, ax = plt.subplots(figsize=(4, 4))
ax.grid(True)
ax.set_title('CC Hamming Loss')
ax.set_xticks(range(0,7))
ax.set_xticklabels(range(0,7))
ax.set_ylabel('Hamming Distance Loss')
ax.set_ylim([0.00, 1.0])
# ax.legend(('Frames Hamming Loss','Video Hamming Loss' ))
plt.plot(frames_mean_hm_test, label='Frames Hamming Loss', color='red')
plt.plot(videos_mean_hm_test, label='Video Hamming Loss', color='green')
plt.legend(loc='upper right')
plt.show()

### ML KNN

In [77]:
from skmultilearn.adapt import MLkNN, MLTSVM
import sklearn.metrics as metrics

## MLTSVM is not compatible with later versions of numpy

kfold = KFold(5, True, 1)
frames_mean_hm_test = []
frames_mean_jac_test = []
videos_mean_jac_test = []
videos_mean_hm_test = []
col_indices = {i:label for (i,label) in enumerate(label_cols)}
best_val = 0
best_model = None
splits = kfold.split(train_videos)
for (i, (train, test)) in enumerate(splits):
    # print('%d-th split: train: %d, validation: %d' % (i+1, len(videos[train]), len(videos[test])))
    train_df = X_df[X_df['filename'].isin(train_videos[train])]
    train_metadata = train_df[metadata_cols]
    print('Training+validation data size: ', train_df.shape[0])
    y_train = train_df[label_cols].values
    X_train = train_df.drop(columns = ['frame', 'face_id', 'culture', 'filename', 'timestamp', 'confidence','success'] + label_cols).values
    valid_df = X_df[X_df['filename'].isin(train_videos[test])]
    y_valid = valid_df[label_cols].values
    X_valid = valid_df.drop(columns = ['frame', 'face_id', 'culture', 'filename', 'timestamp', 'confidence','success']+ label_cols).values
    
    print('Training data size: ', X_train.shape[0])
    print('Validation data size: ', X_valid.shape[0])
    classifier = MLkNN(k=5)
    # classifier = MLTSVM(c_k = 2**-1)
    prediction = classifier.fit(X_train, y_train).predict(X_valid)

    # Predicting on validation set
    val_score = metrics.jaccard_score(y_valid, prediction, average='samples')
    print("Validation Jaccard Score:\n ", val_score)
    if val_score > best_jac:
        best_model = classifier
        best_jac = val_score

# Predicting on test set
y_test_pred = best_model.predict(X_test)

jac_score = jaccard_score(y_test, y_test_pred,  average='samples')
frames_mean_jac_test.append(jac_score)
print("Test Jaccard Score:\n ", jac_score)


# building test dataframe to vote labels
# test_result_df = pd.DataFrame(columns=metadata_cols, data=metadata_test.values)
# test_result_df.update(metadata_test)
# print("^^^^^^^^", y_test_pred.toarray().shape)
temp_df = pd.DataFrame(data=y_test_pred.toarray(), columns=label_cols)
test_result_df = pd.concat([metadata_test, temp_df], axis=1)

# print(test_result_df.head())
video_groups = test_result_df.groupby('filename')[label_cols].apply(lambda x : x.astype(int).sum())
# for name, group in video_groups:
#     print(name)
#     print(group)
#     print("\n") 
ground_truth_video_labels = []
for v, row in video_groups.iterrows():
    # number of 1s in ground truth labels
    ground_truth_video_labels.append(test_df[test_df['filename'] == v].iloc[0][label_cols])
    
    # num_1s = test_df[test_df['filename'] == v].iloc[0][label_cols].sum()
    # num_1s = int(num_1s)
    # # ##### TODO: Bug is somewhere here:
    # a = np.argsort(video_groups.loc[v].values)
    # for i in range(len(a) - 1, len(a) - num_1s - 1, -1):
    #     video_groups.loc[v][a[i]] = 1
    # for i in range(0, len(a) - num_1s):
    #     video_groups.loc[v][a[i]] = 0

    # num_1s = test_df[test_df['filename'] == v].iloc[0][label_cols].sum()
    frames = test_df[test_df['filename'] == v].shape[0]
    # print(num_1s)
    for i in label_ratio.keys():
        # print(type(frames))
        prediction_ratio = row[i]/frames
        # print(prediction_ratio)
        row[i] = 1 if prediction_ratio >= label_ratio[i] else 0


# j = metrics.jaccard_score(np.array(ground_truth_video_labels,  dtype=int), video_groups.values, average='samples')
# h = metrics.hamming_loss(np.array(ground_truth_video_labels,  dtype=int), video_groups.values)
# videos_mean_jac_test.append(j)
# videos_mean_hm_test.append(h)
    

Training+validation data size:  4718
Training data size:  4718
Validation data size:  1253
Validation Jaccard Score:
 
Training+validation data size:  5082
Training data size:  5082
Validation data size:  889
Validation Jaccard Score:
 
Training+validation data size:  4710
Training data size:  4710
Validation data size:  1261
Validation Jaccard Score:
 
Training+validation data size:  4788
Training data size:  4788
Validation data size:  1183
Validation Jaccard Score:
 
Training+validation data size:  4586
Training data size:  4586
Validation data size:  1385
Validation Jaccard Score:
 
Test Jaccard Score:
  0.23521074026529737


In [ ]:
fig, ax = plt.subplots(figsize=(4, 4))
ax.grid(True)
ax.set_title('MLKNN Jaccard Score')
ax.set_xticks(range(0,7))
ax.set_xticklabels(range(0,7))
ax.set_ylabel('Jaccard Similarity Score')
ax.set_ylim([0.00, 1.0])

plt.plot(frames_mean_jac_test, label='Frames Jaccard Score', color='blue')
plt.plot(videos_mean_jac_test, label='Video Jaccard Score', color='green')
plt.legend(loc='best')
plt.show()

In [ ]:
print("F-jac: ", np.mean(frames_mean_jac_test))
print("V-jac: ", np.mean(videos_mean_jac_test))
print("F-ham: ", np.mean(frames_mean_hm_test))
print("V-ham: ", np.mean(videos_mean_hm_test))


In [ ]:
videos_mean_jac_test

In [ ]:
fig, ax = plt.subplots(figsize=(4, 4))
ax.grid(True)
ax.set_title('MLKNN Hamming Loss')
ax.set_xticks(range(0,7))
ax.set_xticklabels(range(0,7))
ax.set_ylabel('Hamming Distance Loss')
ax.set_ylim([0.00, 1.0])
# ax.legend(('Frames Hamming Loss','Video Hamming Loss' ))
plt.plot(frames_mean_hm_test, label='Frames Hamming Loss', color='red')
plt.plot(videos_mean_hm_test, label='Video Hamming Loss', color='green')
plt.legend(loc='upper right')
plt.show()

In [ ]:
np.mean(frames_mean_hm_test)

In [ ]:
train_df.head()

Multilabel confusion matrix puts TN at (0,0) and TP at (1,1) position thanks @Kenneth Witham for pointing out.


`support`: The number of occurrences of each label in y_true.

In [78]:
print('------------------- Frame-level F1 -----------------')
print(metrics.classification_report(y_test, y_test_pred, target_names=label_cols))

print('------------------- Video-level F1 -----------------')
print(metrics.classification_report(np.array(ground_truth_video_labels,  dtype=int),  video_groups.values, target_names=label_cols))

------------------- Frame-level F1 -----------------
              precision    recall  f1-score   support

        none       0.37      0.07      0.11       721
     furious       0.62      0.36      0.46       825
       anger       0.46      0.53      0.49      1355
     annoyed       0.13      0.28      0.18       422
    contempt       0.39      0.18      0.24       630
      hatred       0.47      0.11      0.18      1005

   micro avg       0.39      0.29      0.33      4958
   macro avg       0.41      0.26      0.28      4958
weighted avg       0.44      0.29      0.31      4958
 samples avg       0.38      0.27      0.29      4958

------------------- Video-level F1 -----------------
              precision    recall  f1-score   support

        none       0.00      0.00      0.00         3
     furious       0.67      0.40      0.50         5
       anger       0.55      0.60      0.57        10
     annoyed       0.14      0.40      0.21         5
    contempt       0.67   

In [ ]:
print('------------------- Frame-level Jaccard -----------------')
print(metrics.jaccard_score(y_test, y_test_pred, average='samples'))

print('------------------- Video-level Jaccard -----------------')
print(metrics.jaccard_score(np.array(ground_truth_video_labels,  dtype=int),  video_groups.values, average='samples'))

In [ ]:
print(type(y_test[0]))
print(type(y_test_pred[0].toarray()))
for i in range(len(y_test)):
    if metrics.hamming_loss(y_test[i].flatten(), Y_pred_chains[4][i].flatten()) > 0:
        print("Ground Truth: ", y_test[i], ", Prediction: ", Y_pred_chains[0][i])
        print("Video data: ", metadata_test.iloc[i])

In [ ]:
print(y_test[800:805])
print(metadata_test[800:805])

In [ ]:
print(len(y_test))
print(len(metadata_test))

In [ ]:
def clean(input: str):
    input = 'persian/' + input +".mp4"
    return input
X_df = pd.read_csv('../new_data/Persian/persian_dataset.csv', index_col=None)
X_df['filename'] = X_df['filename'].apply(clean)
X_df.to_csv('../new_data/Persian/persian_dataset.csv', index=False)